<ol>
    <li><a href='#preprocessing'><b>Исследование данных</b></a></li>
    <ul>
        <li><a href="#meet">Приводим в порядок данные</a></li>
        <li><a href="#duplicates">Ищем дубликаты</a></li>
        <li><a href="#conclusion">Промежуточные выводы</a></li>
    </ul>
    <li><a href="#summary"><b>Выводы</b></a></li>
</ol>

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

<a id='preprocessing'></a>
<h1>Исследование</h1>

In [3]:
customer_solvency = pd.read_csv('../datasets/data.csv')

Посмотрим, как выглядит таблица.

In [4]:
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [5]:
print (customer_solvency.columns)

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


Названия столбцов в порядке.

In [6]:
print (customer_solvency.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Количество строк по столбцам не совпадает.<br>В столбцах *'days_employed'* и *'total_income'* не достаёт одинаковое количество строк. В других столбцах количество строк такое же как во всей таблице, то есть пропусков в них нет.<br>Посмотрим долю заполненных значений.<br>И проверим совпадают ли строки с пропусками в *'days_employed'* и *'total_income'*?

In [7]:
print ('Доля пропущенных значений:')
(customer_solvency.isnull().sum() / customer_solvency.shape[0]).apply('{:.1%}'.format)

Доля пропущенных значений:


children             0.0%
days_employed       10.1%
dob_years            0.0%
education            0.0%
education_id         0.0%
family_status        0.0%
family_status_id     0.0%
gender               0.0%
income_type          0.0%
debt                 0.0%
total_income        10.1%
purpose              0.0%
dtype: object

In [8]:
customer_solvency.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [9]:
customer_solvency.tail(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [10]:
customer_solvency.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6641,0,NaN,53,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
17753,0,364580.553256,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,188827.111795,операции с жильем
15763,1,-296.797867,33,среднее,1,женат / замужем,0,M,сотрудник,0,104426.398278,покупка жилья для сдачи
17563,0,-2313.870357,41,среднее,1,в разводе,3,M,сотрудник,0,170889.560954,покупка недвижимости
5305,2,NaN,43,СРЕДНЕЕ,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба


In [11]:
customer_solvency.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [12]:
print(customer_solvency[customer_solvency['days_employed'].isna()]['total_income'].unique())

[nan]


Среди строк с пропусками в столбце *'days_employed'* существуют только строки с пропусками в столбце *'total_income'*, значит в этих строках сосредоточены все пропущенные значения в таблице.<br>Это столбцы с трудовым стажем и доходом, следовательно, можно предположить, что эти люди не работают и никогда не работали, например, студенты.<br>Проверим типы занятости в строках с пропусками.

In [13]:
print(customer_solvency[customer_solvency['days_employed'].isna()]['income_type'].unique())

['пенсионер' 'госслужащий' 'компаньон' 'сотрудник' 'предприниматель']


Ожидаемых безработных и студентов среди неуказавших доход нет.<br>Предположения о пропусках:
- людям было не очень удобно указывать стаж в днях (обычно считают годами)
- месячный доход может быть непостоянным

Наша задача определить зависимость возврата кредита от количества детей и семейного положения, а не от дохода и стажа, так что, думаю, можно заполнить пропуски каким-нибудь числом.<br><br>*Можно было бы принять, что люди со средним образованием начали работать в 18 лет, с высшим в 25 лет и работают по сей день, либо перестали при наступлении пенсионного возраста - все данные для этих рассчётов присутствуют в таблице. Однако предположить доход мы не можем и предположение о количестве дней стажа не выглядит достоверным.*

In [14]:
customer_solvency[customer_solvency['total_income'] == 0]['total_income'].count()

0

<span style="color:green">Нужно число, которым можно заполнить пропуски в столбцах *'days_employed'* и *'total_income'*. 0 - это значение, отсуствие чего-либо, а у нас нет информации об этих людях - они просто не указали ничего. То есть, если бы изначально вместо пропусков стояли нули, это бы означало, что эти люди не работали и не имеют дохода. Предполагаю, что часть из них всё-таки работали и какой-то доход имеют, поэтому нужно числовое значение, означающее, что конкретные данные о стаже и заработки отсуствуют. Так как оба столбца должны принимать положительные значения, логично взять первое отрицательное целое: -1.</span>

In [15]:
customer_solvency['days_employed'] = customer_solvency['days_employed'].fillna(value = -1)
customer_solvency['total_income'] = customer_solvency['total_income'].fillna(value = -1)
print(customer_solvency[customer_solvency['days_employed'].isna()].count())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


***Все пропуски заменены на '-1'.***

In [16]:
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<a id="meet"></a>
## Приводим в порядок данные

Видно, что в столбце со стажем есть отрицательные значения. Проверим сколько отрицательных значений в столбцах, где их быть не должно.

In [17]:
print ('Количество отрицательных значений:')
print ('  children:', customer_solvency['children'][customer_solvency['children'] < 0].count())
print ('  days_employed:', customer_solvency['days_employed'][customer_solvency['days_employed'] < 0].count())
print ('  dob_years:', customer_solvency['dob_years'][customer_solvency['dob_years'] < 0].count())
print ('  total_income:', customer_solvency['total_income'][customer_solvency['total_income'] < 0].count())

Количество отрицательных значений:
  children: 47
  days_employed: 18080
  dob_years: 0
  total_income: 2174


Честно, немного не по себе........<br>
Возможно, такое значение в столбце с количеством детей указали люди возрастом до 18 лет.<br>
Число отрицательных значений в столбцах *days_employed* и *total_income* не совпадают, значит в столбце *days_employed* были отрицательные значения и до замены пропусков на -1.

<div class="alert alert-block alert-info">

Столбец *'children'*

</div>

In [18]:
print (customer_solvency[customer_solvency['children'] < 0]['dob_years'].unique())

[46 50 57 54 55 38 42 26 41 64 32 48 34 51 35 53 31 28 37 63 59 30 27 61
 23 44 40 33 43 69]


Несовершеннолетних нет. Что логично, речь ведь о кредите..<br>Посмотрим, кто указал отрицательное количество детей.

In [19]:
customer_solvency[customer_solvency['children'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,-1.000000,57,Среднее,1,женат / замужем,0,F,пенсионер,0,-1.000000,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Не вижу явных зависимостей. Возможно, какая-то система таким образом заполнила пропуски. Тогда можно считать количество детей для этих людей неизвестным.

In [20]:
print ('{:.3%}'.format(47 / 21525))

0.218%


Так как 47 из 21525 это меньше, чем 0,22%, эти строки можно не учитывать в исследовании. Но перед удалением надо проверить, есть ли у них задолжности по кредиту.

In [21]:
print (customer_solvency['debt'].unique())

[0 1]


В этом столбце отрицательных значений нет, и, более того, значений только два, то есть по сути "есть долг" или "нет долга".<br>Посчитаем сколько людей имеет долг среди тех, кто не указал количество детей, и проверим их семейное положение.

In [22]:
print (customer_solvency[customer_solvency['children'] < 0]['debt'].sum())

1


In [23]:
print (customer_solvency[(customer_solvency['children'] < 0) & (customer_solvency['debt'] == 1)]['family_status'])

16129    женат / замужем
Name: family_status, dtype: object


Один человек - не показатель. Но запомним, что есть человек с долгом по кредиту, который не указал количество детей, и состоит в браке.

In [24]:
print (customer_solvency['children'].unique())

[ 1  0  3  2 -1  4 20  5]


20 детей тоже сомнительное значение.

In [25]:
print (customer_solvency[customer_solvency['children'] > 5]['income_type'].count())

76


In [26]:
print ('{:.3%}'.format(76 / 21525))

0.353%


In [27]:
customer_solvency[customer_solvency['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,-1.000000,53,среднее,1,женат / замужем,0,M,компаньон,0,-1.000000,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Опять-таки не вижу явных зависимостей.<br>*Пробел в таблице ASCII в шестнадцатиричной системе имеет код 20.*<br>Возможно, часть людей, не захотевших указывать количество детей, поставили в этой графе пробел. Тогда можно считать и эту группу людей не указавшими количество детей.

<span style="color:green">Также 20 может быть опечаткой, так как 2 и 0 стоят рядом на num lock, но тогда странно, что не встречается 10 детей, 0 и 1 тоже стоят рядом.<br>Думаю, при анализе можно рассмотреть разные варианты: если 20 это пропуск, если это 0 и, если это 2.</span>

<div class="alert alert-block alert-info">

Столбец *'days_employed'*

</div>

In [28]:
print (customer_solvency['days_employed'].sample(10))

19357     -1534.705969
14081        -1.000000
17989    395750.597112
7058         -1.000000
271      348537.500133
6448      -4172.204302
506       -3551.890732
11098      -494.202990
20584        -1.000000
5209      -1438.617239
Name: days_employed, dtype: float64


Значения достаточно странные для столбца о стаже.<br>Предполагаю, что в анкетах для клиентов можно было указать дату начала работы и дату оконачания, но часть людей всё ещё работает, поэтому даты выхода на пенсию у них нет, а при вычислении разницы система взяла вторым числом время появления unixtime ((00:00:00 UTC) 1 января 1970 года). Тогда для отрицательных значений посчитано количество дней от 1 января 1970 до их начала работы. И тогда стаж можно высчитать "по настоящее время".

In [29]:
print (customer_solvency[customer_solvency['days_employed'] > 0]['income_type'].unique())
print (customer_solvency[customer_solvency['days_employed'] < 0]['income_type'].unique())

['пенсионер' 'безработный']
['сотрудник' 'компаньон' 'пенсионер' 'госслужащий' 'предприниматель'
 'студент' 'в декрете']


Действительно, похоже, что в строках с отрицательными значениями не было даты окончания работы - типы занятости подтверждают предположение.<br>Посмотрим порядок положительных значений.

In [30]:
print (customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'].min())
print (customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'].max())
print (customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'].mean() / 365.25)

328728.72060451825
401755.40047533
999.3273372108642


Добавим столбец с временем начала работы для обработки отрицательных значений.<br>При этом положительные значения пока трогать не будем, потому что они слишком велики, чтобы являться количеством дней работы. Они имеют примерно один и тот же порядок. Но вряд ли эти люди указали стаж в днях, иначе получаются сотни лет работы.

In [31]:
start_date = "1970-01-01"
date_1 = datetime.strptime(start_date, "%Y-%m-%d")
customer_solvency['work_start_day'] = date_1 + abs(customer_solvency[(customer_solvency['days_employed'] <= 0) & (customer_solvency['days_employed'] != -1)]['days_employed']).map(timedelta)

customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,work_start_day
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,1993-02-06 16:09:09.598484
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,1981-01-07 19:17:24.332679
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,1985-05-25 10:08:33.523955
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1981-04-17 17:55:58.645058
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,1982-05-27 07:35:59.699013
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,NaT
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,1975-10-15 08:19:31.097084
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,1978-07-10 11:33:39.328922


1. Вероятно, дата окончания работы действительно не указана, но число дней изначально записано верно, поскольку, если это число дней от 1 января 1970, как мы предположили ранее, то 27-летний `[5]` начал работать раньше, чем 42-летний `[0]`, а если посчитать, то это невозможно. То есть для отрицательный хначений нужен стаж по модулю и без дробной части. Столбец *'work_start_day'* можно удалить.
2. Образовались пропуски в новом столбце в строках с положительными значениями. Разберёмся, что это за числа.

In [32]:
del customer_solvency['work_start_day']
print ((customer_solvency[customer_solvency['days_employed'] > 0]['dob_years'] -
        customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'] / 365.25 / 24).head(15))

4     14.183428
18     7.337082
24    18.378970
25    25.527437
30    23.717811
35    23.051212
50    22.647338
56    21.774916
71    23.428984
78    19.963844
86    20.201968
87    14.480774
88    20.434430
98    12.372552
99   -39.532469
dtype: float64


Если стаж указан в часах, то тоже получаются не очень реальные числа: люди должны были начать работать до 12 лет, а в некоторых случаях до рождения.

Если поделить на 40, получается тот же порядок, что у отрицательных. 40 - это количество рабочих часов в неделю. Если стаж считался в днях по 40 часов в неделю, то здесь записано количество часов работы.

In [33]:
print ((customer_solvency[customer_solvency['days_employed'] > 0]['dob_years'] -
        customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'] /40/365.25).sample(15))

# количество людей с изначально положительным значением стажа, чей предполагаемый возраст начала работы до рождения
count_0 = ((customer_solvency[customer_solvency['days_employed'] > 0]['dob_years'] -
            customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'] /40/365.25) < 0).count()
print ('Количество отрицательных значений:', count_0)

# количество людей с изначально положительным значением стажа,
# которые предположительно начали работать до 12 лет
count_12 = (((customer_solvency[customer_solvency['days_employed'] > 0]['dob_years'] -
         customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'] / 40 / 365.25) < 12).count() -
        ((customer_solvency[customer_solvency['days_employed'] > 0]['dob_years'] -
         customer_solvency[customer_solvency['days_employed'] > 0]['days_employed'] / 40 / 365.25) < 0).count())
print ('Количество тех, кто начал работать до 12 лет:', count_12)

print ('Сомнительных значений в сумме:', count_0 + count_12)

12935    30.948061
20997    33.111579
6656     36.040597
4055     32.216379
17522    43.514577
13569    37.955898
1087     30.340904
12631    36.315523
14379    42.240877
385      25.385890
14337    37.304006
5947     37.199509
13500    37.702097
11993    36.418620
4958     37.032223
dtype: float64
Количество отрицательных значений: 3445
Количество тех, кто начал работать до 12 лет: 0
Сомнительных значений в сумме: 3445


In [34]:
print ('{:.3%}'.format((count_0 + count_12) / customer_solvency.shape[0]))

16.005%


Добавим новый столбец со стажем в днях.<br>Положительные значения перепишем разделив на 40, отрицательные по модулю.

In [35]:
customer_solvency.loc[(customer_solvency['days_employed'] <= 0) & (customer_solvency['days_employed'] != -1), 'new_days_employed'] = abs(customer_solvency['days_employed'])
customer_solvency.loc[customer_solvency['days_employed'] == -1, 'new_days_employed'] = customer_solvency['days_employed']
customer_solvency.loc[customer_solvency['days_employed'] > 0, 'new_days_employed'] = customer_solvency['days_employed'] / 40

customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437.673028
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024.803754
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623.422610
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,8506.651801
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,4529.316663
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,8598.435103
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,2113.346888
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,3112.481705


Тип данных вещественный, но так как стаж нужен в днях, дробную часть в любом случае можно отбросить.

In [36]:
customer_solvency['new_days_employed'] = customer_solvency['new_days_employed'].astype('int')
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,8437
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4024
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,5623
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,4124
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,8506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,4529
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,8598
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,2113
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,3112


In [37]:
print ('Отрицательных значений:', customer_solvency[customer_solvency['new_days_employed'] < 0]['new_days_employed'].count())
new_count_12 = customer_solvency[(customer_solvency['dob_years'] - customer_solvency['new_days_employed'] / 365.25) < 12]['new_days_employed'].count()
print ('Тех, кто работает дольше, чем с 12 лет:', new_count_12)

Отрицательных значений: 2174
Тех, кто работает дольше, чем с 12 лет: 130


Отрицательные значения остались только те, которые мы установили на -1 как неизвестные.

<div class="alert alert-block alert-info">

Столбец *'total_income'*

</div>

Приведём в порядок остальные столбцы.<br>Доход в месяц *'total_income'* так же не интересует нас с точностью до копейки.<br>Преобразуем тип в целочисленный.

In [38]:
customer_solvency['total_income'] = customer_solvency['total_income'].astype('int')
print (customer_solvency['total_income'].sample(5))

15654    144147
9362     219371
246      158350
9389     108855
9652     130760
Name: total_income, dtype: int32


Посмотрим, какие данные там записаны.

In [39]:
print ('Количество пропусков:', customer_solvency[customer_solvency['total_income'].isna()]['total_income'].count())
print ('Минимальное значение:', customer_solvency['total_income'].min())
print ('Максимальное значение:', customer_solvency['total_income'].max())
print ('Среднее значение:', customer_solvency['total_income'].mean())
print ('Медиана', customer_solvency['total_income'].median())

Количество пропусков: 0
Минимальное значение: -1
Максимальное значение: 2265604
Среднее значение: 150512.2931939605
Медиана 135514.0


<div class="alert alert-block alert-info">

Столбцы *'education'* и *'family_status'*

</div>

Переведём столбцы *'education'* и *'family_status'* в нижний регистр.<br>
*в столбце 'family_status' значение "не женат" написано с большой буквы*

In [40]:
customer_solvency['education'] = customer_solvency['education'].str.lower()
customer_solvency['family_status'] = customer_solvency['family_status'].str.lower()
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,8506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4529
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,8598
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,2113
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3112


Проверим соответствие значений столбцов индексам этих значений.

In [41]:
print (customer_solvency.groupby(by = 'education')['education_id'].unique())

education
высшее                 [0]
начальное              [3]
неоконченное высшее    [2]
среднее                [1]
ученая степень         [4]
Name: education_id, dtype: object


In [42]:
print (customer_solvency.groupby(by = 'family_status')['family_status_id'].unique())

family_status
в разводе                [3]
вдовец / вдова           [2]
гражданский брак         [1]
женат / замужем          [0]
не женат / не замужем    [4]
Name: family_status_id, dtype: object


<div class="alert alert-block alert-info">

Столбец *'purpose'*

</div>

In [43]:
print (customer_solvency['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Есть явные совпадения в разных формулировках, например, 'сыграть свадьбу' и 'на проведение свадьбы'.<br>
Лемматизируем.<br>

In [44]:
purposes = customer_solvency.purpose.unique()

def lemmater(text):
    lemmas = m.lemmatize(text)
    lemmas = ''.join(lemmas)
    return lemmas[:-1]

lemmas_purposes = [''] * len(purposes)
for i in range(len(purposes)):
    lemmas_purposes[i] = lemmater(purposes[i])

for i in range(len(lemmas_purposes)):
    print (lemmas_purposes[i])

покупка жилье
приобретение автомобиль
дополнительный образование
сыграть свадьба
операция с жилье
образование
на проведение свадьба
покупка жилье для семья
покупка недвижимость
покупка коммерческий недвижимость
покупка жилой недвижимость
строительство собственный недвижимость
недвижимость
строительство недвижимость
на покупка подержать автомобиль
на покупка свой автомобиль
операция с коммерческий недвижимость
строительство жилой недвижимость
жилье
операция со свой недвижимость
автомобиль
заниматься образование
сделка с подержанный автомобиль
получение образование
автомобиль
свадьба
получение дополнительный образование
покупка свой жилье
операция с недвижимость
получение высокий образование
свой автомобиль
сделка с автомобиль
профильный образование
высокий образование
покупка жилье для сдача
на покупка автомобиль
ремонт жилье
заниматься высокий образование


Получается большой список разных слов. Оставим только существительные для категоризации.<br>Так же будем обращать внимание на слова "жилой" и "сдача": жилая недвижимость в дальнейшем попадёт в категорию "жилье", жилье для сдачи - "недвижимость".<br>Предполагаю, что покупка, строительство, приобретение стоят дороже, чем ремонт или различные операции с чем-либо. То есть кредит больше и вероятность возврата может отличаться. Значит надо ещё обращать внимание и на эти слова.

In [45]:
purpose_s = [''] * len(lemmas_purposes)
addition_word_purchase = ['покупка', 'приобретение', 'строительство']
addition_word_procedure = ['операция', 'ремонт']

for i in range(len(lemmas_purposes)):
    if 'сдача' in lemmas_purposes[i].split():
        purpose_s[i] = 'приобретение недвижимость '
    elif 'жилой' in lemmas_purposes[i].split() and 'недвижимость' in lemmas_purposes[i].split():
        purpose_s[i] = 'приобретение жилье '
    else:
        for word in lemmas_purposes[i].split():
            if m.analyze(word)[0]['analysis'][0]['gr'][0] == 'S':
                if word in addition_word_purchase:
                    purpose_s[i] += 'приобретение' + ' '
                elif word in addition_word_procedure:
                    purpose_s[i] += 'операция' + ' '
                else:
                    purpose_s[i] += m.analyze(word)[0]['analysis'][0]['lex'] + ' '
            
for i in range(len(purpose_s)):
    purpose_s[i] = purpose_s[i][:-1]
    print (purpose_s[i])

приобретение жилье
приобретение автомобиль
образование
свадьба
операция с жилье
образование
проведение свадьба
приобретение жилье семья
приобретение недвижимость
приобретение недвижимость
приобретение жилье
приобретение недвижимость
недвижимость
приобретение недвижимость
приобретение автомобиль
приобретение автомобиль
операция с недвижимость
приобретение жилье
жилье
операция недвижимость
автомобиль
образование
сделка с автомобиль
получение образование
автомобиль
свадьба
получение образование
приобретение жилье
операция с недвижимость
получение образование
автомобиль
сделка с автомобиль
образование
образование
приобретение недвижимость
приобретение автомобиль
операция жилье
образование


Можно выделить 7 категорий:

In [46]:
purposes_type = ['автомобиль', 'образование', 'свадьба', 'приобретение недвижимости',
                 'приобретение жилья', 'операции с недвижимостью', 'операции с жильем']

*Под недвижимостью понимается коммерческая недвижимость.*

Запишем в отдельный столбец причины взятия кредита в этих категориях.<br>
- Лемматизируем причины всего столбца.
- Запишем в новый столбец одну из 7 выделенных категорий.

Цикл работает очень долго для DataFrame. Лемматизация вынесена в отдельную функцию ```lemmater```,  но и с apply() работает очень долго. Сформируем новый столбец с лемматизированными причинами через словарь, созданный по уникальным значениям столбца *'purpose'*.

In [47]:
lemmas_map = dict(zip(purposes, purpose_s))
customer_solvency['lemmas'] = customer_solvency['purpose'].map(lemmas_map)
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed,lemmas
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,приобретение жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,приобретение автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,приобретение жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,8506,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4529,операция с жилье
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,8598,сделка с автомобиль
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,2113,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3112,приобретение автомобиль


In [48]:
uniq_purpose = list(set(purpose_s))

for i in range(len(uniq_purpose)):
    print (uniq_purpose[i])

свадьба
недвижимость
автомобиль
операция жилье
операция недвижимость
приобретение жилье
сделка с автомобиль
получение образование
операция с недвижимость
операция с жилье
образование
проведение свадьба
приобретение автомобиль
жилье
приобретение жилье семья
приобретение недвижимость


Так как неизвестно, что предполагается, когда написано просто "недвижимость" или "жилье", будем предполагать приобретение исходя из идеи, что обычно говорят "кредит на жилье", когда хотят его купить, то есть приобрести.

In [49]:
lemmas_purposes_type = ['автомобиль', 'образование', 'свадьба', 'приобретение недвижимость',
                 'приобретение жилье', 'операция с недвижимость', 'операция с жилье']
categore = [''] * len(uniq_purpose)

for i in range(len(uniq_purpose)):
    for j in range(len(lemmas_purposes_type)):
        if lemmas_purposes_type[j] in uniq_purpose[i]:
            categore[i] = lemmas_purposes_type[j]
        elif 'жилье' == uniq_purpose[i]:
            categore[i] = 'приобретение жилье'
        elif 'операция жилье' == uniq_purpose[i]:
            categore[i] = 'операция с жилье'
        elif 'недвижимость' == uniq_purpose[i]:
            categore[i] = 'приобретение недвижимость'
        elif 'операция недвижимость' == uniq_purpose[i]:
            categore[i] = 'операция с недвижимость'
            

lemmas_purpose_map = dict(zip(uniq_purpose, categore))
customer_solvency['categor'] = customer_solvency['lemmas'].map(lemmas_purpose_map)
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed,lemmas,categor
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,приобретение жилье,приобретение жилье
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,приобретение автомобиль,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,приобретение жилье,приобретение жилье
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,8506,свадьба,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4529,операция с жилье,операция с жилье
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,8598,сделка с автомобиль,автомобиль
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,2113,недвижимость,приобретение недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3112,приобретение автомобиль,автомобиль


In [50]:
print (customer_solvency['categor'].unique())

['приобретение жилье' 'автомобиль' 'образование' 'свадьба'
 'операция с жилье' 'приобретение недвижимость' 'операция с недвижимость']


Всё записалось правильно.

И последний штрих - преобразование категорий причин взятия кредита в человеческую форму. Выставим индексы для категорий.<br>После чего вспомогательные столбцы *'lemmas'* и *'categor'* можно удалить.

In [51]:
categor_purpose_map = dict(zip(lemmas_purposes_type, purposes_type))
customer_solvency['purpose_type'] = customer_solvency['categor'].map(categor_purpose_map)
categor_purpose_map_id = dict(zip(purposes_type, range(len(purposes_type))))
customer_solvency['purpose_type_id'] = customer_solvency['purpose_type'].map(categor_purpose_map_id)
del customer_solvency['lemmas']
del customer_solvency['categor']
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed,purpose_type,purpose_type_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,приобретение жилья,4
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,автомобиль,0
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,приобретение жилья,4
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,8506,свадьба,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4529,операции с жильем,6
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,8598,автомобиль,0
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,2113,приобретение недвижимости,3
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3112,автомобиль,0


In [52]:
print (customer_solvency.groupby(by = 'purpose_type')['purpose_type_id'].unique())

purpose_type
автомобиль                   [0]
образование                  [1]
операции с жильем            [6]
операции с недвижимостью     [5]
приобретение жилья           [4]
приобретение недвижимости    [3]
свадьба                      [2]
Name: purpose_type_id, dtype: object


<a id="duplicates"></a>
## Ищем дубликаты

Столбца, однозначно идентифицирующего конкретного человека, нет.<br>Ищем абсолютные совпадения.

In [53]:
customer_solvency.duplicated().sum()

71

In [54]:
customer_solvency[customer_solvency.duplicated() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed,purpose_type,purpose_type_id
2849,0,-1.0,41,среднее,1,женат / замужем,0,F,сотрудник,0,-1,покупка жилья для семьи,-1,приобретение жилья,4
3290,0,-1.0,58,среднее,1,гражданский брак,1,F,пенсионер,0,-1,сыграть свадьбу,-1,свадьба,2
4182,1,-1.0,34,высшее,0,гражданский брак,1,F,сотрудник,0,-1,свадьба,-1,свадьба,2
4851,0,-1.0,60,среднее,1,гражданский брак,1,F,пенсионер,0,-1,свадьба,-1,свадьба,2
5557,0,-1.0,58,среднее,1,гражданский брак,1,F,пенсионер,0,-1,сыграть свадьбу,-1,свадьба,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,-1.0,64,среднее,1,женат / замужем,0,F,пенсионер,0,-1,дополнительное образование,-1,образование,1
21032,0,-1.0,60,среднее,1,женат / замужем,0,F,пенсионер,0,-1,заняться образованием,-1,образование,1
21132,0,-1.0,47,среднее,1,женат / замужем,0,F,сотрудник,0,-1,ремонт жилью,-1,операции с жильем,6
21281,1,-1.0,30,высшее,0,женат / замужем,0,F,сотрудник,0,-1,покупка коммерческой недвижимости,-1,приобретение недвижимости,3


In [55]:
print (customer_solvency[customer_solvency.duplicated() == True]['new_days_employed'].unique())
print (customer_solvency[customer_solvency.duplicated() == True]['total_income'].unique())
print (customer_solvency[customer_solvency.duplicated() == True]['debt'].unique())

[-1]
[-1]
[0]


Все дубликаты, это строки, в которых люди не указали стаж и заработок, задолжностей по кредитам у них нет. Возможно, эти люди пришли брать кредит в первый раз, подумали, что заполнили что-то неправильно или что-то не сохранилось и заполнили ещё раз, хотя всё сохранилось. Тогда можно просто удалить эти строки.

In [56]:
customer_solvency = customer_solvency.drop_duplicates().reset_index(drop = True)
customer_solvency

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed,purpose_type,purpose_type_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,8437,приобретение жилья,4
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,4024,автомобиль,0
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5623,приобретение жилья,4
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4124,образование,1
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,8506,свадьба,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4529,операции с жильем,6
21450,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,8598,автомобиль,0
21451,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,2113,приобретение недвижимости,3
21452,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,3112,автомобиль,0


In [57]:
customer_solvency.duplicated().sum()

0

<a id="conclusion"></a>
## Промежуточные выводы

In [58]:
children_debt_sum = pd.Series(customer_solvency.groupby(by = 'children')['debt'].sum())
children_debt_percentage_group = pd.Series(customer_solvency.groupby(by = 'children')['debt'].sum() / customer_solvency.groupby(by = 'children')['children'].count()).apply('{:.2%}'.format)
children_percentage = pd.Series(customer_solvency.groupby(by = 'children')['children'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённым количеством детей:', children_debt_sum)
print ('Процент должников среди людей с тем же количеством детей:', children_debt_percentage_group)
print ('Процент людей с определённым количеством детей:', children_percentage)

Количество должников с определённым количеством детей: children
-1        1
 0     1063
 1      444
 2      194
 3       27
 4        4
 5        0
 20       8
Name: debt, dtype: int64
Процент должников среди людей с тем же количеством детей: children
-1      2.13%
 0      7.54%
 1      9.23%
 2      9.45%
 3      8.18%
 4      9.76%
 5      0.00%
 20    10.53%
dtype: object
Процент людей с определённым количеством детей: children
-1      0.22%
 0     65.68%
 1     22.41%
 2      9.56%
 3      1.54%
 4      0.19%
 5      0.04%
 20     0.35%
Name: children, dtype: object


Из всех людей больше всего должников не имеет детей. Но людей, не имеющих детей, просто больше всего. А если рассматривать процент должников среди людей с одинаковым количеством детей, получается, что только люди с пятью детьми не имеют долгов. Наобильший процент должников наблюдается среди тех, кто имеет 4 детей.

Предполагаю, что 4 ребёнка это тяжело и просрочить выплаты или в принципе не накопить достаточное для возврата кредита количество денег не так сложно в такой ситуации. Однако, среди тех, кто имеет 5 детей, должников нет. Возможно, они максимально ответственны, раз решили завести 5 детей.<br>В оставшихся группах люди с 1 или 2 детьми реже возвращают кредиты, чем те, у кого детей нет или не указавших информации о детях. Возможно, иметь 1 или 2 детей достаточно обыденная ситуация, "как у всех", и на финансовую ответственность не влияет.

<span style="color:green">Рассмотрим варианты с 20 детьми.</span>

1. Если 20 это пропуск.

In [59]:
customer_solvency.loc[customer_solvency['children'] != 20, 'new_children'] = customer_solvency['children']
customer_solvency.loc[customer_solvency['children'] == 20, 'new_children'] = -1

In [60]:
children_debt_sum_20 = pd.Series(customer_solvency.groupby(by = 'new_children')['debt'].sum())
children_debt_percentage_group_20 = pd.Series(children_debt_sum_20 / customer_solvency.groupby(by = 'new_children')['new_children'].count()).apply('{:.2%}'.format)
children_percentage_20 = pd.Series(customer_solvency.groupby(by = 'new_children')['new_children'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённым количеством детей:', children_debt_sum_20)
print ('Процент должников среди людей с тем же количеством детей:', children_debt_percentage_group_20)
print ('Процент людей с определённым количеством детей:', children_percentage_20)

Количество должников с определённым количеством детей: new_children
-1.0       9
 0.0    1063
 1.0     444
 2.0     194
 3.0      27
 4.0       4
 5.0       0
Name: debt, dtype: int64
Процент должников среди людей с тем же количеством детей: new_children
-1.0    7.32%
 0.0    7.54%
 1.0    9.23%
 2.0    9.45%
 3.0    8.18%
 4.0    9.76%
 5.0    0.00%
dtype: object
Процент людей с определённым количеством детей: new_children
-1.0     0.57%
 0.0    65.68%
 1.0    22.41%
 2.0     9.56%
 3.0     1.54%
 4.0     0.19%
 5.0     0.04%
Name: new_children, dtype: object


Если считать значение 20 как пропуск, то те, кто не захотел указывать количество детей, и те, кто их не имеет, примерно одинаково возвращают кредиты.<br>Наибольший и наименьший проценты должников остались в тех же группах.

2. Если 20 это 0.

In [61]:
customer_solvency.loc[customer_solvency['children'] != 20, 'new_children'] = customer_solvency['children']
customer_solvency.loc[customer_solvency['children'] == 20, 'new_children'] = 0

In [62]:
children_debt_sum_0 = pd.Series(customer_solvency.groupby(by = 'new_children')['debt'].sum())
children_debt_percentage_group_0 = pd.Series(children_debt_sum_0 / customer_solvency.groupby(by = 'new_children')['new_children'].count()).apply('{:.2%}'.format)
children_percentage_0 = pd.Series(customer_solvency.groupby(by = 'new_children')['new_children'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённым количеством детей:', children_debt_sum_0)
print ('Процент должников среди людей с тем же количеством детей:', children_debt_percentage_group_0)
print ('Процент людей с определённым количеством детей:', children_percentage_0)

Количество должников с определённым количеством детей: new_children
-1.0       1
 0.0    1071
 1.0     444
 2.0     194
 3.0      27
 4.0       4
 5.0       0
Name: debt, dtype: int64
Процент должников среди людей с тем же количеством детей: new_children
-1.0    2.13%
 0.0    7.56%
 1.0    9.23%
 2.0    9.45%
 3.0    8.18%
 4.0    9.76%
 5.0    0.00%
dtype: object
Процент людей с определённым количеством детей: new_children
-1.0     0.22%
 0.0    66.03%
 1.0    22.41%
 2.0     9.56%
 3.0     1.54%
 4.0     0.19%
 5.0     0.04%
Name: new_children, dtype: object


Если считать значение 20 как опечатку, означающую 0, то наибольший и наименьший проценты должников остаются в тех же группах, что и изначально, но среди людей, не указавших количество детей, должников меньше, чем среди тех, у кого до 5 детей.

3. Если 20 это 2.

In [63]:
customer_solvency.loc[customer_solvency['children'] != 20, 'new_children'] = customer_solvency['children']
customer_solvency.loc[customer_solvency['children'] == 20, 'new_children'] = 2

In [64]:
children_debt_sum_2 = pd.Series(customer_solvency.groupby(by = 'new_children')['debt'].sum())
children_debt_percentage_group_2 = pd.Series(children_debt_sum_2 / customer_solvency.groupby(by = 'new_children')['new_children'].count()).apply('{:.2%}'.format)
children_percentage_2 = pd.Series(customer_solvency.groupby(by = 'new_children')['new_children'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённым количеством детей:', children_debt_sum_2)
print ('Процент должников среди людей с тем же количеством детей:', children_debt_percentage_group_2)
print ('Процент людей с определённым количеством детей:', children_percentage_2)

Количество должников с определённым количеством детей: new_children
-1.0       1
 0.0    1063
 1.0     444
 2.0     202
 3.0      27
 4.0       4
 5.0       0
Name: debt, dtype: int64
Процент должников среди людей с тем же количеством детей: new_children
-1.0    2.13%
 0.0    7.54%
 1.0    9.23%
 2.0    9.49%
 3.0    8.18%
 4.0    9.76%
 5.0    0.00%
dtype: object
Процент людей с определённым количеством детей: new_children
-1.0     0.22%
 0.0    65.68%
 1.0    22.41%
 2.0     9.92%
 3.0     1.54%
 4.0     0.19%
 5.0     0.04%
Name: new_children, dtype: object


Если считать значение 20 как опечатку, означающую 2, то все соотношения остаются теми же.

In [65]:
print ('Процент должников среди тех, кто указал 20 детей: {:.2%}'.format( 
       customer_solvency[customer_solvency['children'] == 20]['debt'].sum() /
        customer_solvency[customer_solvency['children'] == 20]['children'].count()))

print ('Процент должников, которые указали 20 детей: {:.2%}'.format( 
       customer_solvency[customer_solvency['children'] == 20]['debt'].sum() /
        customer_solvency['debt'].sum()))

Процент должников среди тех, кто указал 20 детей: 10.53%
Процент должников, которые указали 20 детей: 0.46%


Из всех должников доля тех, кто указал 20 детей, составляет 0.46%, что очень мало.

In [66]:
del customer_solvency['new_children']

Количество детей с 3, 4 и 5 детьми очень маленькое, чтобы рассматривать каждую группу в отдельности. Посмотрим количество должников среди тех, у кого детей больше 2.

In [67]:
children_group_345 = customer_solvency[(customer_solvency['children'] > 2) & (customer_solvency['children'] < 20)]

children_debt_sum = children_group_345['debt'].sum()
children_debt_percentage_group = children_group_345['debt'].sum() / children_group_345['children'].count()
children_percentage = children_group_345['children'].count() / customer_solvency.shape[0]
print ('Количество должников, имеющих больше 2 детей:', children_debt_sum)
print ('Процент должников среди людей, имеющих больше 2 детей: {:.2%}'.format(children_debt_percentage_group))
print ('Процент людей, имеющих больше 2 детей: {:.2%}'.format(children_percentage))

Количество должников, имеющих больше 2 детей: 31
Процент должников среди людей, имеющих больше 2 детей: 8.16%
Процент людей, имеющих больше 2 детей: 1.77%


In [68]:
family_status_debt_sum = pd.Series(customer_solvency.groupby(by = 'family_status')['debt'].sum())
family_status_debt_percentage_group = pd.Series(customer_solvency.groupby(by = 'family_status')['debt'].sum() / customer_solvency.groupby(by = 'family_status')['family_status'].count()).apply('{:.2%}'.format)
family_status_percentage = pd.Series(customer_solvency.groupby(by = 'family_status')['family_status'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённым семейным положением:', family_status_debt_sum)
print ('\nПроцент должников среди людей с тем же семейным положением:', family_status_debt_percentage_group)
print ('\nПроцент людей с определённым семейным положением:', family_status_percentage)

Количество должников с определённым семейным положением: family_status
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
не женат / не замужем    274
Name: debt, dtype: int64

Процент должников среди людей с тем же семейным положением: family_status
в разводе                7.11%
вдовец / вдова           6.57%
гражданский брак         9.35%
женат / замужем          7.55%
не женат / не замужем    9.75%
dtype: object

Процент людей с определённым семейным положением: family_status
в разводе                 5.57%
вдовец / вдова            4.47%
гражданский брак         19.35%
женат / замужем          57.51%
не женат / не замужем    13.10%
Name: family_status, dtype: object


Из всех людей больше всего должников женаты или замужем. Среди людей с одинаковым семейным положением наибольший процент должников наблюдается у тех, кто не женат или состоит в гражданском браке.

Опять-таки, гражданский брак - ситуация достаточно стандартная. А вот вдовцов, видимо, радости жизни меньше отвлекают от возвратов кредитов.

In [69]:
percentil = [0, .25, .5, .75, 1]
percentil_name = ['0%', '25%', '50%', '75%', '100%']
customer_solvency[(customer_solvency['total_income'] > -1)]['total_income'].describe(percentil)

count    1.935100e+04
mean     1.674218e+05
std      1.029716e+05
min      2.066700e+04
0%       2.066700e+04
25%      1.030530e+05
50%      1.450170e+05
75%      2.034345e+05
100%     2.265604e+06
max      2.265604e+06
Name: total_income, dtype: float64

In [70]:
min_total_income = customer_solvency[(customer_solvency['total_income'] > -1)]['total_income'].describe(percentil)[percentil_name[0]]
total_income_25 = customer_solvency[(customer_solvency['total_income'] > -1)]['total_income'].describe(percentil)[percentil_name[1]]
total_income_50 = customer_solvency[(customer_solvency['total_income'] > -1)]['total_income'].describe(percentil)[percentil_name[2]]
total_income_75 = customer_solvency[(customer_solvency['total_income'] > -1)]['total_income'].describe(percentil)[percentil_name[3]]
max_total_income = customer_solvency[(customer_solvency['total_income'] > -1)]['total_income'].describe(percentil)[percentil_name[4]]

customer_solvency.loc[customer_solvency['total_income'] == -1, 'income_percentil'] = 'Не указавшие доход'
customer_solvency.loc[(customer_solvency['total_income'] >= min_total_income) & (customer_solvency['total_income'] < total_income_25), 'income_percentil'] = 'От 0 до '+str(total_income_25)
customer_solvency.loc[(customer_solvency['total_income'] >= total_income_25) & (customer_solvency['total_income'] < total_income_50), 'income_percentil'] = 'От '+str(total_income_25)+' до '+str(total_income_50)
customer_solvency.loc[(customer_solvency['total_income'] >= total_income_50) & (customer_solvency['total_income'] < total_income_75), 'income_percentil'] = 'От '+str(total_income_50)+' до '+str(total_income_75)
customer_solvency.loc[customer_solvency['total_income'] >= total_income_75, 'income_percentil'] = 'От '+str(total_income_75)


customer_solvency.groupby(by = 'income_percentil')['debt'].sum()
income_debt_sum = pd.Series(customer_solvency.groupby(by = 'income_percentil')['debt'].sum())
income_debt_percentage_group = pd.Series(customer_solvency.groupby(by = 'income_percentil')['debt'].sum() / customer_solvency.groupby(by = 'income_percentil')['income_percentil'].count()).apply('{:.2%}'.format)
income_percentage = pd.Series(customer_solvency.groupby(by = 'income_percentil')['income_percentil'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённой зарплатой:', income_debt_sum)
print ('\nПроцент должников среди людей с той же зарплатой:', income_debt_percentage_group)
print ('\nПроцент людей с определённой зарплатой:', income_percentage)

Количество должников с определённой зарплатой: income_percentil
Не указавшие доход         170
От 0 до 103053.0           383
От 103053.0 до 145017.0    421
От 145017.0 до 203434.5    426
От 203434.5                341
Name: debt, dtype: int64

Процент должников среди людей с той же зарплатой: income_percentil
Не указавшие доход         8.08%
От 0 до 103053.0           7.92%
От 103053.0 до 145017.0    8.70%
От 145017.0 до 203434.5    8.81%
От 203434.5                7.05%
dtype: object

Процент людей с определённой зарплатой: income_percentil
Не указавшие доход          9.80%
От 0 до 103053.0           22.55%
От 103053.0 до 145017.0    22.55%
От 145017.0 до 203434.5    22.55%
От 203434.5                22.55%
Name: income_percentil, dtype: object


Среди людей с одинаковым уровнем дохода наименьший процент должников наблюдается у тех, кто зарабатывает больше, чем 75% людей, то есть больше 203434.5.<br>Возможно, они зарабатывают достаточно, чтобы не копить/не откладывать на возврат кредита.<br>Но среди людей с предыдущем уровнем дохода наблюдается, наоборот, наибольший процент должников.<br>Верхняя граница дохода этой группы достаточно близка к 200 000. Возможно, существует некоторый психологический барьер, что доход вроде нормальный, но это ещё не "кгрулая" сумма, и расставаться с деньгами сложнее.

In [71]:
purpose_type_debt_sum = pd.Series(customer_solvency.groupby(by = 'purpose_type')['debt'].sum())
purpose_type_debt_percentage_group = pd.Series(customer_solvency.groupby(by = 'purpose_type')['debt'].sum() / customer_solvency.groupby(by = 'purpose_type')['purpose_type'].count()).apply('{:.2%}'.format)
purpose_type_percentage = pd.Series(customer_solvency.groupby(by = 'purpose_type')['purpose_type'].count() / customer_solvency.shape[0]).apply('{:.2%}'.format)
print ('Количество должников с определённой причиной кредита:', purpose_type_debt_sum)
print ('\nПроцент должников среди людей с той же причиной кредита:', purpose_type_debt_percentage_group)
print ('\nПроцент людей с определённой причиной кредита:', purpose_type_percentage)

Количество должников с определённой причиной кредита: purpose_type
автомобиль                   403
образование                  370
операции с жильем             83
операции с недвижимостью     157
приобретение жилья           262
приобретение недвижимости    280
свадьба                      186
Name: debt, dtype: int64

Процент должников среди людей с той же причиной кредита: purpose_type
автомобиль                   9.36%
образование                  9.22%
операции с жильем            6.59%
операции с недвижимостью     8.04%
приобретение жилья           6.93%
приобретение недвижимости    7.33%
свадьба                      8.00%
dtype: object

Процент людей с определённой причиной кредита: purpose_type
автомобиль                   20.07%
образование                  18.71%
операции с жильем             5.87%
операции с недвижимостью      9.10%
приобретение жилья           17.62%
приобретение недвижимости    17.81%
свадьба                      10.83%
Name: purpose_type, dtype: object


Из всех людей больше всего должников берут кредит на автомобиль. Но и людей, которые бредут кредит на автомобиль, больше всего. Среди людей с одинаковыми причинами взятия кредита наибольший процент должников наблюдается у тех, кто берёт кредит на автомобиль и на образование.

Предполагаю, что люди, которые берут кредит на образование, рассчитывают свои расходы не совсем верно, думая, что получив образование начнут зарабатывать сразу и много, чего не происходит.<br>Возможно, те, кто берёт кредит на автомобиль, так же рассчитывают на больший доход в дальнейшем. Например, автомобиль позволит им работать на более высокоплачиваемой работе, до которой слишком времязатратно добираться общественным транспортом.

<a id="summary"></a>
# Выводы
В изначальных данных наблюдался ряд проблем, которые могли повлиять на качество исследования.<br>
1. Столбец *total_income (доход в месяц)* имел пропуски в 10.1% случаев.
2. Столбец *children (количество детей)* имеет значение `-1` в 0.22% случаев, что было расценено как пропуски. Среди этих людей нашёлся только 1 должник. Это замужняя женщина 33 лет со средним образованием, имеющая доход 149641, которая брала кредит на автомобиль.
3. Столбец *children (количество детей)* имеет значение `20` в 0.35% случаев. Это можно расценить как пропуск, отсутствие детей (0) или двоих детей (2).
4. Столбец *days_employed (стаж в днях)* имел отрицательные не целочисленные значения разного порядка по модулю. Было выдвинуто предположение о том, что это связано с обработкой анкет: отрицательные числа указаны у тех, кто работает по сей день, а стаж пенсионеров посчитан по рабочим часам, то есть 40 в неделю. В соответствии с этим предположением образовалось 130 людей, начавших работать до достижения 12-летнего возраста.
5. Причины взятия кредитов *purpose* указаны в свободной форме. При обработке этого столбца было выделено 7 категорий: автомобиль, образование, свадьба, приобретение недвижимости, приобретение жилья, операции с недвижимостью, операции с жильем. При этом предполагается, что 'покупка', 'приобретение' и 'строительство' требуют больший размер кредита, чем 'ремонт' и другие 'операции', поэтому они разделены на 2 категории. Также 'недвижимость' (и 'сдача жилья') предполагается коммерческой, а 'жилая недвижимость' жильём.

<br>Вероятность возврата кредита мало зависит от количества детей, семейного положения, уровня дохода и причины взятия кредита. Количество должников в каждой группе отличается буквально на 1-2 процента.


<br>Тех, кто имеет 3, 4 или 5 детей достаточно мало, будем рассматривать их как одну группу.

Количество детей|20 как пропуск|20 как 0|20 как 2
-|-|-|-
не указано|7.32%|2.13%|2.13%
0.0|7.54%|7.56%|7.54%
1.0|9.23%|9.23%|9.23%
2.0|9.45%|9.45%|9.49%
> 2|8.16%|8.16%|8.16%

Наибольший процент должников: имеют 2 детей
<br>Наименьший процент должников: не имеют детей или не указали их количество


Семейное положение|Процент должников
-|-
в разводе|7.11%
вдовец / вдова|6.57%
гражданский брак|9.35%
женат / замужем|7.55%
не женат / не замужем|9.75%

Наибольший процент должников: находятся в гражданском браке
<br>Наименьший процент должников: являются вдовцом / вдовой

<br>Все люди без учёта тех, кто не указал свой заработок, были разделены на равные группы в зависимости от уровня дохода.

Уровень дохода|Процент должников
-|-
Не указавшие доход|8.08%
От 0 до 103053.0|7.92%
От 103053.0 до 145017.0|8.70%
От 145017.0 до 203434.5|8.81%
От 203434.5|7.05%

Наибольший процент должников: получают от 145017.0 до 203434.5 в месяц
<br>Наименьший процент должников: получают больше 203434.5 в месяц


Причина взятия кредита|Процент должников
-|-
автомобиль|9.36%
образование|9.22%
операции с жильем|6.59%
операции с недвижимостью|8.04%
приобретение жилья|6.93%
приобретение недвижимости|7.33%
свадьба|8.00%

Наибольший процент должников: кредиты на автомобиль и образование
<br>Наименьший процент должников: кредит на жильё

<br>Возможные причины таких показателей:
1. 2 ребёнка достаточно распространённая ситуация, то есть "как у всех", и на финансовую ответственность не влияет. А отсутствие детей позволяет проще планировать своё бюджет.
2. Гражданский брак так же достаточно обыденная ситуация, а вот вдовцов, по всей видимости, мало что отвлекает от возвратов кредитов.
3. Люди, чей заработок немного не дотягивает до "круглой" суммы в 200 000, скорее всего тяжелее расстаются с деньгами, чем те, кто зарабатывает больше 200 000.
4. Люди, которые берут кредиты на автомобиль и образование, скорее всего рассчитывают на приток дохода из-за реализации причины кредита, чего, по всей видимости, в большинстве случаев не происходит.

<br>Также, вероятно, стоит внимательнее отнестись к понятию "гражданский брак" и ситуации с детьми - оба ли взрослых человека работают, сможет ли партнёр взять на себя непредвиденные расходы, чтобы заёмщик мог продолжить выплачивать кредит.
<br>Возможно, у тех, кто зарабатывает "почти" 200 000, стоит узнать степень занятости и предложить автоматические выплаты с гибкой системой, например, не раз в месяц, а раз в неделю или, наоборот, раз в квартал. Может быть, они много работают, чтобы перейти эту черту в 200 000 и просто забывают или не успевают.
<br>Возможно, имеет смысл обращать внимание на стаж на конкретной работе, где человек и получает эту зарплату, предполагая, сколько он там проработает с таким уровнем дохода, чтобы не оказалось, что лучшей работы не появилось, а денег не хватает.

<br>Думаю, наиболее информативными окажутся сочетания признаков, например, количество должников, у которых 2 ребёнка после развода.